<a href="https://colab.research.google.com/github/x1001000/Colab-Notebooks/blob/master/%E4%BA%9E%E9%A6%AC%E9%81%9C%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 點下左側 Files 裡的 Mount Drive，第一次會要求授權
# 下 Linux 指令在雲端硬碟新增資料夾
! mkdir drive/My\ Drive/amazon.co.jp

In [ ]:
# 匯入所需的 Python 套件
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 網站有防爬蟲，藉此偽裝成 Mac 電腦上的瀏覽器
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

In [ ]:
keyword = input('請輸入商品關鍵字：')
print('\n網站一頁有60筆')
pages = int(input('請問要爬幾頁？'))

In [ ]:
print('商品資料下載彙整中（一頁約兩分鐘）。。。\n')
df = pd.DataFrame(columns=['產品名稱', '價格', '商品介紹', '網頁連結'])
for p in range(pages):
    r = requests.get(f'https://www.amazon.co.jp/s?k={keyword}&page={p}', headers=headers)
    soup = BeautifulSoup(r.text)
    products = soup.find_all(class_= 'a-link-normal a-text-normal')
    for product in products:
        df.loc[len(df)+1] = None
        df.iloc[-1]['產品名稱'] = product.text.strip('\n')
        df.iloc[-1]['網頁連結'] = 'https://www.amazon.co.jp'+product.get('href')
        r = requests.get(df.iloc[-1]['網頁連結'], headers=headers)
        soup = BeautifulSoup(r.text)
        if soup.find(class_ = 'a-unordered-list a-vertical a-spacing-mini'):
            df.iloc[-1]['商品介紹'] = soup.find(class_ = 'a-unordered-list a-vertical a-spacing-mini').text.strip('\n')
        else:
            print(df.iloc[-1]['產品名稱'])
            print('無商品介紹')
            df.iloc[-1]['商品介紹'] = '無商品介紹'
        if soup.find(id = 'priceblock_saleprice'):
            df.iloc[-1]['價格'] = soup.find(id = 'priceblock_saleprice').text.strip('\n')
        elif soup.find(id = 'priceblock_ourprice'):
            df.iloc[-1]['價格'] = soup.find(id = 'priceblock_ourprice').text.strip('\n')
        else:
            print(df.iloc[-1]['產品名稱'])
            print('無價格')
            df.iloc[-1]['價格'] = '無價格'
df.to_csv('drive/My Drive/amazon.co.jp/'+keyword+'.csv')
print('已匯出 file.csv 👉 左側 Files 👉 上方 Refresh 👉 右鍵 Download 或到 Google Drive 直接開啟')